In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [37]:
data = pd.read_csv('data/train.csv')
print(f"shape= {data.shape}")
data.head(10)

shape= (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [38]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [51]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,2:], data.iloc[:,1:2], test_size=0.20, random_state=0)

In [52]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

((712, 10), (179, 10), (712, 1), (179, 1))

In [53]:
X_train['Age'].fillna(X_train['Age'].median(skipna=True), inplace=True)
X_train['Embarked'].fillna(X_train['Embarked'].value_counts().idxmax(), inplace=True)

In [54]:
X_train.drop(['Cabin', 'Name', 'Ticket'], axis=1, inplace=True)
X_test.drop(['Cabin', 'Name', 'Ticket'], axis=1, inplace=True)

In [55]:
X_test['Age'].fillna(X_train['Age'].median(skipna=True), inplace=True)
X_test['Embarked'].fillna(X_train['Embarked'].value_counts().idxmax(), inplace=True)

In [56]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

((712, 7), (179, 7), (712, 1), (179, 1))

In [57]:
X_train.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [61]:
X_train.dtypes

Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object

In [62]:
import category_encoders as ce
encoder = ce.OrdinalEncoder(cols=['Sex', 'Embarked'])
X_train = encoder.fit_transform(X_train)
X_test = encoder.fit_transform(X_test)

In [69]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [74]:
param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [10, 50, 100],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 6],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'reg_alpha': [0.1, 10, 0.01]
}


xgboost = xgb.XGBClassifier()


grid_search = GridSearchCV(xgboost, param_grid, cv=5)
grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_
best_score = grid_search.best_score_

In [75]:
best_params ,best_score 

({'gamma': 0.2,
  'learning_rate': 0.1,
  'max_depth': 5,
  'min_child_weight': 1,
  'n_estimators': 10,
  'reg_alpha': 0.01,
  'subsample': 0.8},
 0.8356544863587118)

In [77]:
model = xgb.XGBClassifier(
    learning_rate=0.1,
    max_depth=5,
    min_child_weight=1,
    n_estimators=10,
    reg_alpha=0.01,
    gamma=0.2,
    subsample=0.8
)

In [78]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.2, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=10, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [80]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, model.predict(X_test))

0.40782122905027934